In [ ]:
#Download Required Libraries
!pip install -q transformers datasets
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

# #Load Required Libraries

In [ ]:

from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Load Pretrained Model and Tokenizer

In [ ]:
# Load Pretrained Model and Tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ✅ Fix: Set pad_token to eos_token for padding compatibility
tokenizer.pad_token = tokenizer.eos_token







/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Chat
input_text = "User: I'm feeling a bit anxious today. <|sep|> Bot:"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

User: I'm feeling a bit anxious today. <|sep|> Bot: I'm feeling anxious too.


# Prepare Data (Format: prompt-response pairs)

In [ ]:
# Prepare Data (Format: prompt-response pairs)
# Example format:
# "User: I feel really anxious today. <|sep|> Bot: I'm sorry you're feeling that way. I'm here if you need to talk."


dataset = load_dataset("nbertagnolli/counsel-chat", download_mode="force_redownload")


def build_prompt_response(example):
    if example["questionText"] is None or example["answerText"] is None:
        return {"input_text": "", "response_text": ""}  # or skip later
    prompt = f"User: {example['questionText'].strip()} <|sep|> Bot:"
    response = example['answerText'].strip()
    return {"input_text": prompt, "response_text": response}


formatted_dataset = dataset["train"].map(build_prompt_response)

README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


20220401_counsel_chat.csv:   0%|          | 0.00/4.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

Map:   0%|          | 0/2775 [00:00<?, ? examples/s]

In [ ]:
'''training for another data
dataset = load_dataset("facebook/empathetic_dialogues", download_mode="force_redownload")


def build_prompt_response(example):
    if example["context"] is None or example["utterance"] is None:
        return {"input_text": "", "response_text": ""}  # or skip later
    prompt = f"User: {example['context'].strip()} <|sep|> Bot:"
    response = example['utterance'].strip()
    return {"input_text": prompt, "response_text": response}


formatted_dataset = dataset["train"].map(build_prompt_response)'''


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

The repository for facebook/empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

# Tokenize

In [ ]:
# Tokenize
def tokenize_function(example):
    input_ids = tokenizer.encode(example["input_text"] + " " + example["response_text"], truncation=True, padding="max_length", max_length=128)
    return {"input_ids": input_ids, "labels": input_ids}

tokenized_dataset = formatted_dataset.map(tokenize_function)

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

# Fine-Tuning Setup

In [ ]:
# Fine-Tuning Setup

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,        # ↓ reduce if you run into memory issues
    gradient_accumulation_steps=4,        # ↑ effective batch size = 2 × 4 = 8
    num_train_epochs=1,                   # ↑ more epochs for deeper learning
    learning_rate=5e-5,
    #warmup_steps=100,                     # ↑ helps stabilize early training
    weight_decay=0.1,                    # ↑ helps regularization
    logging_dir="./logs",
    logging_steps=20,                     # ↓ reduce noise
    save_strategy="epoch",               # ↑ save model at each epoch
    fp16=True,                            # ↑ speed up training if on GPU
    save_total_limit=2                    # ↓ only keep last 2 checkpoints
)



'''
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no"
)'''

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


#trainer.train_dataset = tokenized_dataset.select(range(200))

# Train

In [4]:
# Train
trainer.train() #using T4 GPU #1040	1.689300'''



'## Step 6: Train\ntrainer.train() #using T4 GPU #1040\t1.689300'

# Inference

In [ ]:
# Inference
input_text = "User: I've been feeling very down lately. <|sep|> Bot:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(trainer.model.device)
#output = trainer.model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)

output = trainer.model.generate(
    input_ids,
    max_length=200,
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=3,         # Prevents repeating 3-grams
    repetition_penalty=1.5,         # Penalizes reused tokens
    top_p=0.92,                     # Nucleus sampling (soft randomness)
    top_k=50,                       # Limits to top 50 tokens
    do_sample=True,                 # Enables sampling instead of greedy
    temperature=0.7                 # Controls randomness (lower = safer)
)



print(tokenizer.decode(output[0], skip_special_tokens=True))

# Save to file
#output_path = Path("/mnt/data/mental_health_chatbot_dialoGPT.py")
#output_path.write_text(notebook_content.strip())

#output_path

User: I've been feeling very down lately. <|sep|> Bot: It's understandable that you have been feeling down recently!  There are some things to keep in mind, and perhaps a few others which may help you feel better.  One thing I would recommend is to take a look at what your definition of "down" is when something feels out of balance - such as how you describe being unable to function without sufficient support or energy from others around you?  Something like this could be a symptom of depression, anxiety, anger, or even grief.  If it's possible for you to bring up the issue


# Save model and tokenizer

In [ ]:
tokenizer.save_pretrained("/content/model")
trainer.model.save_pretrained("/content/model")

# Demo on streamlit

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
#tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")
#model = AutoModelForCausalLM.from_pretrained("/content/model")

model_name = "yhlien1221/dialoggpt-finetune-counsellor-data"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/77.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
# Define your prompt
user_input = "I've been feeling very anxious and overwhelmed lately."
input_text = f"User: {user_input} <|sep|> Bot:"

# Step 5: Tokenize input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Optional: Move to GPU if available
if torch.cuda.is_available():
    model = model.to("cuda")
    input_ids = input_ids.to("cuda")

# Step 6: Generate response
output_ids = model.generate(
    input_ids,
    max_length=150,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

# Step 7: Decode and show result
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
bot_reply = response.split("<|sep|> Bot:")[-1].strip()
print(f"🤖 Bot: {bot_reply}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤖 Bot: Hello, This is a very common feeling among individuals who are dealing with depression.  I can see how it can feel overwhelming.  You may be feeling overwhelmed because you have been so overwhelmed with anxiety.  It is normal for people to feel anxious when they are overwhelmed with anxiety.  This is normal and normal for people to feel overwhelmed when they are overwhelmed with anxiety.  This is also normal for people who are overwhelmed with anxiety when they are overwhelmed by anxiety.  There are many ways to be overwhelmed with anxiety.  I


In [6]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model & Tokenizer from Hugging Face
model_name = "yhlien1221/dialoggpt-finetune-counsellor-data"

@st.cache_resource
def load_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model_and_tokenizer()

st.title("🧠 Mental Health Counselor Chatbot")
st.caption("🤖 A fine-tuned DialoGPT model for supportive conversation.")

# Initialize chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Display previous messages
for role, message in st.session_state.chat_history:
    with st.chat_message(role):
        st.markdown(message)

# Chat input
if prompt := st.chat_input("You:"):
    # Show user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Build full prompt from history
    full_prompt = ""
    for role, message in st.session_state.chat_history:
        if role == "user":
            full_prompt += f"User: {message} <|sep|> "
        elif role == "bot":
            full_prompt += f"Bot: {message} <|sep|> "
    full_prompt += f"User: {prompt} <|sep|> Bot:"

    # Tokenize and generate
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt")

    output_ids = model.generate(
        input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2
    )

    # Decode and extract response
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    reply = decoded.split("<|sep|> Bot:")[-1].strip()

    # Show and store bot reply
    with st.chat_message("bot"):
        st.markdown(reply)

    st.session_state.chat_history.append(("user", prompt))
    st.session_state.chat_history.append(("bot", reply))


Overwriting app.py


In [8]:
!npm install localtunnel
!streamlit run app.py --server.address=localhost &>/content/logs.txt &
!npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

⠙⠹⠸
up to date, audited 23 packages in 922ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼34.106.88.181⠙your url is: https://fast-results-dance.loca.lt
